<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [2]</a>'.</span>

# 📊 Exploratory Data Analysis (EDA)
## Hotel Booking Cancellation Prediction

**Mục tiêu:**
1. Hiểu cấu trúc và đặc điểm của dữ liệu
2. Phân tích phân phối các biến
3. Tìm hiểu mối quan hệ giữa các biến với target `is_canceled`
4. Phát hiện vấn đề: missing values, outliers, data leakage

**Dataset:** Hotel Booking Demand - Kaggle

## 1. Import Required Libraries

In [1]:
# Standard libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Statistical tests
from scipy import stats
from scipy.stats import chi2_contingency

# Custom modules
import sys
sys.path.append('..')
from src.data.loader import (
    load_raw_data, 
    print_data_summary, 
    get_column_types,
    NUMERICAL_COLUMNS,
    CATEGORICAL_COLUMNS,
    TARGET_COLUMN,
    LEAKAGE_COLUMNS
)

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

# Plot settings
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['font.size'] = 11

print("✓ Libraries imported successfully!")

✓ Libraries imported successfully!


## 2. Load Dataset

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [2]:
# Load data using custom loader module
df = load_raw_data(file_path='../data/raw/hotel_bookings.csv')

# Display first few rows
df.head()

FileNotFoundError: Data file not found: ../data/raw/hotel_bookings.csv

## 3. Overview Statistics

### 3.1. Basic Info

In [ ]:
# Use custom summary function
print_data_summary(df)

In [ ]:
# Detailed info
print("="*50)
print("DETAILED INFO")
print("="*50)
df.info()

### 3.2. Descriptive Statistics (Numerical)

In [ ]:
# Descriptive statistics for numerical columns
df.describe().T

### 3.3. Categorical Variables Summary

In [ ]:
# Categorical columns summary
cat_cols = df.select_dtypes(include=['object']).columns.tolist()
print(f"Số cột categorical: {len(cat_cols)}\n")

for col in cat_cols:
    print(f"--- {col} ---")
    print(f"Unique values: {df[col].nunique()}")
    print(df[col].value_counts().head(5))
    print()

## 4. Missing Values Analysis

In [ ]:
# Calculate missing values
missing_df = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum().values,
    'Missing_Percentage': (df.isnull().sum() / len(df) * 100).values,
    'Data_Type': df.dtypes.values
})

# Sort by missing percentage
missing_df = missing_df.sort_values('Missing_Percentage', ascending=False)
missing_df = missing_df[missing_df['Missing_Count'] > 0]

print("="*60)
print("MISSING VALUES SUMMARY")
print("="*60)
print(f"\nTổng số cột có missing: {len(missing_df)}")
print(f"Tổng số missing values: {df.isnull().sum().sum():,}")
print()
missing_df

In [ ]:
# Visualize missing values
fig, ax = plt.subplots(figsize=(10, 5))

colors = ['#ff6b6b' if pct > 50 else '#feca57' if pct > 10 else '#48dbfb' 
          for pct in missing_df['Missing_Percentage']]

bars = ax.barh(missing_df['Column'], missing_df['Missing_Percentage'], color=colors)
ax.set_xlabel('Missing Percentage (%)')
ax.set_title('Missing Values by Column', fontsize=14, fontweight='bold')

# Add percentage labels
for bar, pct in zip(bars, missing_df['Missing_Percentage']):
    ax.text(bar.get_width() + 1, bar.get_y() + bar.get_height()/2, 
            f'{pct:.2f}%', va='center', fontsize=10)

plt.tight_layout()
plt.savefig('../outputs/figures/missing_values.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n⚠️ Lưu ý:")
print("- company: 94.31% missing → Nên DROP")
print("- agent: 13.69% missing → Xử lý (fill với 0 hoặc 'Unknown')")
print("- country: 0.41% missing → Fill với mode")
print("- children: 0.00% missing → Fill với 0")

## 5. Data Dictionary and Variable Classification

### 5.1. Data Dictionary

In [ ]:
# Data Dictionary
data_dict = {
    'Column': [
        'hotel', 'is_canceled', 'lead_time', 'arrival_date_year', 'arrival_date_month',
        'arrival_date_week_number', 'arrival_date_day_of_month', 'stays_in_weekend_nights',
        'stays_in_week_nights', 'adults', 'children', 'babies', 'meal', 'country',
        'market_segment', 'distribution_channel', 'is_repeated_guest', 'previous_cancellations',
        'previous_bookings_not_canceled', 'reserved_room_type', 'assigned_room_type',
        'booking_changes', 'deposit_type', 'agent', 'company', 'days_in_waiting_list',
        'customer_type', 'adr', 'required_car_parking_spaces', 'total_of_special_requests',
        'reservation_status', 'reservation_status_date'
    ],
    'Description': [
        'Loại khách sạn (Resort Hotel / City Hotel)',
        '⭐ TARGET: Booking có bị huỷ không (0=Không, 1=Có)',
        'Số ngày từ khi đặt đến ngày nhận phòng',
        'Năm nhận phòng (2015-2017)',
        'Tháng nhận phòng',
        'Số tuần trong năm của ngày nhận phòng',
        'Ngày trong tháng của ngày nhận phòng',
        'Số đêm cuối tuần (Thứ 7, Chủ nhật)',
        'Số đêm trong tuần (Thứ 2-6)',
        'Số người lớn',
        'Số trẻ em',
        'Số em bé',
        'Loại bữa ăn đã đặt',
        'Quốc gia của khách',
        'Phân khúc thị trường (Online TA, Offline TA/TO, Direct, etc.)',
        'Kênh phân phối (TA/TO, Direct, Corporate, etc.)',
        'Khách có phải là khách quay lại không (0/1)',
        'Số lần huỷ booking trước đó',
        'Số lần booking trước đó không bị huỷ',
        'Loại phòng đã đặt',
        'Loại phòng được gán (có thể khác với đặt)',
        'Số lần thay đổi booking',
        'Loại đặt cọc (No Deposit, Non Refund, Refundable)',
        'ID của đại lý du lịch',
        'ID của công ty đặt phòng',
        'Số ngày booking nằm trong danh sách chờ',
        'Loại khách hàng (Transient, Contract, Group, etc.)',
        'Average Daily Rate - Giá phòng trung bình mỗi đêm',
        'Số chỗ đỗ xe yêu cầu',
        'Số yêu cầu đặc biệt',
        '⚠️ LEAKAGE: Trạng thái cuối cùng (Check-Out/Canceled/No-Show)',
        '⚠️ LEAKAGE: Ngày cập nhật trạng thái'
    ],
    'Type': [
        'Categorical', 'Target (Binary)', 'Numerical', 'Numerical', 'Categorical',
        'Numerical', 'Numerical', 'Numerical', 'Numerical', 'Numerical',
        'Numerical', 'Numerical', 'Categorical', 'Categorical', 'Categorical',
        'Categorical', 'Binary', 'Numerical', 'Numerical', 'Categorical',
        'Categorical', 'Numerical', 'Categorical', 'Numerical', 'Numerical',
        'Numerical', 'Categorical', 'Numerical', 'Numerical', 'Numerical',
        'LEAKAGE', 'LEAKAGE'
    ]
}

data_dict_df = pd.DataFrame(data_dict)
data_dict_df.index = range(1, len(data_dict_df) + 1)
data_dict_df

### 5.2. Variable Classification

In [ ]:
# Get column types from loader module
col_types = get_column_types(df)

print("="*60)
print("VARIABLE CLASSIFICATION")
print("="*60)
print(f"\n🎯 TARGET: {col_types['target']}")
print(f"\n📊 NUMERICAL ({len(col_types['numerical'])} columns):")
for i, col in enumerate(col_types['numerical'], 1):
    print(f"   {i}. {col}")

print(f"\n📋 CATEGORICAL ({len(col_types['categorical'])} columns):")
for i, col in enumerate(col_types['categorical'], 1):
    print(f"   {i}. {col}")

print(f"\n⚠️ LEAKAGE ({len(col_types['leakage'])} columns):")
for col in col_types['leakage']:
    print(f"   - {col}")

## 6. Target Variable Distribution (is_canceled)

**Kiểm tra Imbalance của biến target**

In [ ]:
# Target distribution
target_counts = df['is_canceled'].value_counts()
target_pct = df['is_canceled'].value_counts(normalize=True) * 100

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar plot
colors = ['#2ecc71', '#e74c3c']
bars = axes[0].bar(['Not Canceled (0)', 'Canceled (1)'], target_counts.values, color=colors)
axes[0].set_ylabel('Count')
axes[0].set_title('Target Distribution - Count', fontsize=12, fontweight='bold')

# Add labels
for bar, count, pct in zip(bars, target_counts.values, target_pct.values):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1000, 
                 f'{count:,}\n({pct:.1f}%)', ha='center', fontsize=11)

# Pie chart
axes[1].pie(target_counts.values, labels=['Not Canceled', 'Canceled'], 
            colors=colors, autopct='%1.1f%%', startangle=90,
            explode=(0, 0.05), shadow=True)
axes[1].set_title('Target Distribution - Percentage', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig('../outputs/figures/target_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

# Summary
print("="*50)
print("TARGET DISTRIBUTION SUMMARY")
print("="*50)
print(f"Not Canceled (0): {target_counts[0]:,} ({target_pct[0]:.2f}%)")
print(f"Canceled (1):     {target_counts[1]:,} ({target_pct[1]:.2f}%)")
print(f"\nImbalance Ratio: 1:{target_counts[0]/target_counts[1]:.2f}")
print("\n⚠️ Dataset có IMBALANCED (37% vs 63%)")
print("→ Cần xử lý: SMOTE / class_weight / PR-AUC metric")

## 7. Lead Time Distribution Analysis

`lead_time`: Số ngày từ khi đặt phòng đến ngày nhận phòng - Đây là feature quan trọng!

In [ ]:
# Lead time analysis
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Overall histogram
axes[0, 0].hist(df['lead_time'], bins=50, color='steelblue', edgecolor='white', alpha=0.7)
axes[0, 0].set_xlabel('Lead Time (days)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Lead Time Distribution', fontsize=12, fontweight='bold')
axes[0, 0].axvline(df['lead_time'].median(), color='red', linestyle='--', label=f'Median: {df["lead_time"].median():.0f}')
axes[0, 0].axvline(df['lead_time'].mean(), color='orange', linestyle='--', label=f'Mean: {df["lead_time"].mean():.0f}')
axes[0, 0].legend()

# 2. Boxplot
axes[0, 1].boxplot(df['lead_time'], vert=True)
axes[0, 1].set_ylabel('Lead Time (days)')
axes[0, 1].set_title('Lead Time Boxplot', fontsize=12, fontweight='bold')

# 3. Lead time by cancellation status
df_not_canceled = df[df['is_canceled'] == 0]['lead_time']
df_canceled = df[df['is_canceled'] == 1]['lead_time']

axes[1, 0].hist(df_not_canceled, bins=50, alpha=0.6, label=f'Not Canceled (mean={df_not_canceled.mean():.0f})', color='green')
axes[1, 0].hist(df_canceled, bins=50, alpha=0.6, label=f'Canceled (mean={df_canceled.mean():.0f})', color='red')
axes[1, 0].set_xlabel('Lead Time (days)')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_title('Lead Time by Cancellation Status', fontsize=12, fontweight='bold')
axes[1, 0].legend()

# 4. Violin plot
violin_data = [df_not_canceled, df_canceled]
vp = axes[1, 1].violinplot(violin_data, positions=[0, 1], showmeans=True, showmedians=True)
vp['bodies'][0].set_facecolor('green')
vp['bodies'][1].set_facecolor('red')
axes[1, 1].set_xticks([0, 1])
axes[1, 1].set_xticklabels(['Not Canceled', 'Canceled'])
axes[1, 1].set_ylabel('Lead Time (days)')
axes[1, 1].set_title('Lead Time Violin Plot by Status', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig('../outputs/figures/lead_time_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

# Statistics
print("="*50)
print("LEAD TIME STATISTICS")
print("="*50)
print(f"\nOverall:")
print(f"  Mean: {df['lead_time'].mean():.1f} days")
print(f"  Median: {df['lead_time'].median():.0f} days")
print(f"  Std: {df['lead_time'].std():.1f} days")
print(f"  Min: {df['lead_time'].min()} days")
print(f"  Max: {df['lead_time'].max()} days")

print(f"\nBy Cancellation Status:")
print(f"  Not Canceled - Mean: {df_not_canceled.mean():.1f} days")
print(f"  Canceled - Mean: {df_canceled.mean():.1f} days")
print(f"\n💡 Insight: Booking bị huỷ có lead_time cao hơn đáng kể!")
print(f"   → Lead time dài = Rủi ro huỷ cao hơn")

## 8. Cancellation Rate by Hotel Type

In [ ]:
# Cancellation by hotel type
hotel_cancel = df.groupby('hotel')['is_canceled'].agg(['sum', 'count'])
hotel_cancel['cancel_rate'] = hotel_cancel['sum'] / hotel_cancel['count'] * 100
hotel_cancel.columns = ['Canceled', 'Total', 'Cancel_Rate']

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 1. Stacked bar chart
hotel_crosstab = pd.crosstab(df['hotel'], df['is_canceled'])
hotel_crosstab.plot(kind='bar', stacked=True, ax=axes[0], color=['#2ecc71', '#e74c3c'])
axes[0].set_xlabel('Hotel Type')
axes[0].set_ylabel('Count')
axes[0].set_title('Bookings by Hotel Type and Cancellation Status', fontsize=12, fontweight='bold')
axes[0].legend(['Not Canceled', 'Canceled'])
axes[0].tick_params(axis='x', rotation=0)

# 2. Cancellation rate bar chart
colors = ['#3498db', '#9b59b6']
bars = axes[1].bar(hotel_cancel.index, hotel_cancel['Cancel_Rate'], color=colors)
axes[1].set_xlabel('Hotel Type')
axes[1].set_ylabel('Cancellation Rate (%)')
axes[1].set_title('Cancellation Rate by Hotel Type', fontsize=12, fontweight='bold')

for bar, rate in zip(bars, hotel_cancel['Cancel_Rate']):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
                 f'{rate:.1f}%', ha='center', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig('../outputs/figures/hotel_type_cancellation.png', dpi=150, bbox_inches='tight')
plt.show()

# Summary table
print("="*60)
print("CANCELLATION BY HOTEL TYPE")
print("="*60)
print(hotel_cancel.to_string())
print(f"\n💡 Insight: City Hotel có tỷ lệ huỷ cao hơn Resort Hotel")

## 9. Cancellation Rate by Month and Season

In [ ]:
# Create season mapping
season_map = {
    'December': 'Winter', 'January': 'Winter', 'February': 'Winter',
    'March': 'Spring', 'April': 'Spring', 'May': 'Spring',
    'June': 'Summer', 'July': 'Summer', 'August': 'Summer',
    'September': 'Fall', 'October': 'Fall', 'November': 'Fall'
}

# Month order for plotting
month_order = ['January', 'February', 'March', 'April', 'May', 'June',
               'July', 'August', 'September', 'October', 'November', 'December']

df['season'] = df['arrival_date_month'].map(season_map)

# Calculate cancellation rate by month
monthly_cancel = df.groupby('arrival_date_month')['is_canceled'].mean() * 100
monthly_cancel = monthly_cancel.reindex(month_order)

# Calculate cancellation rate by season
season_order = ['Winter', 'Spring', 'Summer', 'Fall']
season_cancel = df.groupby('season')['is_canceled'].mean() * 100
season_cancel = season_cancel.reindex(season_order)

fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# 1. Monthly cancellation rate
axes[0].plot(range(12), monthly_cancel.values, marker='o', linewidth=2, markersize=8, color='#e74c3c')
axes[0].fill_between(range(12), monthly_cancel.values, alpha=0.3, color='#e74c3c')
axes[0].set_xticks(range(12))
axes[0].set_xticklabels([m[:3] for m in month_order], rotation=45)
axes[0].set_xlabel('Month')
axes[0].set_ylabel('Cancellation Rate (%)')
axes[0].set_title('Cancellation Rate by Month', fontsize=12, fontweight='bold')
axes[0].axhline(y=df['is_canceled'].mean()*100, color='gray', linestyle='--', label='Average')
axes[0].legend()

# 2. Seasonal cancellation rate
colors = ['#3498db', '#2ecc71', '#f1c40f', '#e67e22']
bars = axes[1].bar(season_cancel.index, season_cancel.values, color=colors)
axes[1].set_xlabel('Season')
axes[1].set_ylabel('Cancellation Rate (%)')
axes[1].set_title('Cancellation Rate by Season', fontsize=12, fontweight='bold')

for bar, rate in zip(bars, season_cancel.values):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
                 f'{rate:.1f}%', ha='center', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.savefig('../outputs/figures/monthly_seasonal_cancellation.png', dpi=150, bbox_inches='tight')
plt.show()

# Summary
print("="*60)
print("MONTHLY & SEASONAL CANCELLATION RATES")
print("="*60)
print("\nBy Month:")
for month, rate in monthly_cancel.items():
    print(f"  {month}: {rate:.1f}%")

print("\nBy Season:")
for season, rate in season_cancel.items():
    print(f"  {season}: {rate:.1f}%")

print(f"\n💡 Insight: Mùa hè có tỷ lệ huỷ cao nhất")

## 10. Cancellation Rate by Market Segment

In [ ]:
# Cancellation by market segment
segment_stats = df.groupby('market_segment').agg({
    'is_canceled': ['sum', 'count', 'mean']
}).round(4)
segment_stats.columns = ['Canceled', 'Total', 'Cancel_Rate']
segment_stats['Cancel_Rate'] = segment_stats['Cancel_Rate'] * 100
segment_stats = segment_stats.sort_values('Cancel_Rate', ascending=True)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 1. Horizontal bar chart - Cancellation rate
colors = plt.cm.RdYlGn_r(np.linspace(0.2, 0.8, len(segment_stats)))
bars = axes[0].barh(segment_stats.index, segment_stats['Cancel_Rate'], color=colors)
axes[0].set_xlabel('Cancellation Rate (%)')
axes[0].set_title('Cancellation Rate by Market Segment', fontsize=12, fontweight='bold')
axes[0].axvline(x=df['is_canceled'].mean()*100, color='red', linestyle='--', label='Average')

for bar, rate in zip(bars, segment_stats['Cancel_Rate']):
    axes[0].text(bar.get_width() + 1, bar.get_y() + bar.get_height()/2, 
                 f'{rate:.1f}%', va='center', fontsize=10)

# 2. Booking volume by segment
segment_counts = df['market_segment'].value_counts()
axes[1].pie(segment_counts.values, labels=segment_counts.index, autopct='%1.1f%%', 
            startangle=90, colors=plt.cm.Set3.colors[:len(segment_counts)])
axes[1].set_title('Booking Volume by Market Segment', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig('../outputs/figures/market_segment_cancellation.png', dpi=150, bbox_inches='tight')
plt.show()

# Summary
print("="*60)
print("CANCELLATION BY MARKET SEGMENT")
print("="*60)
print(segment_stats.to_string())
print(f"\n💡 Insights:")
print("- Groups có tỷ lệ huỷ cao nhất")
print("- Direct bookings có tỷ lệ huỷ thấp nhất")
print("- Online TA chiếm số lượng lớn nhất")

## 11. Cancellation Rate by Country (Top 10)

In [ ]:
# Top 10 countries by booking volume
top_countries = df['country'].value_counts().head(10).index.tolist()

# Filter for top countries
df_top_countries = df[df['country'].isin(top_countries)]

# Calculate stats
country_stats = df_top_countries.groupby('country').agg({
    'is_canceled': ['sum', 'count', 'mean']
})
country_stats.columns = ['Canceled', 'Total', 'Cancel_Rate']
country_stats['Cancel_Rate'] = country_stats['Cancel_Rate'] * 100
country_stats = country_stats.sort_values('Total', ascending=False)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 1. Booking volume vs Cancellation rate
x = np.arange(len(country_stats))
width = 0.35

ax1 = axes[0]
bars1 = ax1.bar(x - width/2, country_stats['Total'], width, label='Total Bookings', color='steelblue')
ax1.set_xlabel('Country')
ax1.set_ylabel('Total Bookings', color='steelblue')
ax1.set_xticks(x)
ax1.set_xticklabels(country_stats.index, rotation=45)
ax1.tick_params(axis='y', labelcolor='steelblue')

ax2 = ax1.twinx()
ax2.plot(x, country_stats['Cancel_Rate'], 'ro-', linewidth=2, markersize=8, label='Cancel Rate')
ax2.set_ylabel('Cancellation Rate (%)', color='red')
ax2.tick_params(axis='y', labelcolor='red')

ax1.set_title('Top 10 Countries: Bookings & Cancellation Rate', fontsize=12, fontweight='bold')
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

# 2. Cancellation rate sorted
country_by_rate = country_stats.sort_values('Cancel_Rate', ascending=True)
colors = plt.cm.RdYlGn_r(np.linspace(0.2, 0.8, len(country_by_rate)))
bars = axes[1].barh(country_by_rate.index, country_by_rate['Cancel_Rate'], color=colors)
axes[1].set_xlabel('Cancellation Rate (%)')
axes[1].set_title('Cancellation Rate by Country (Top 10)', fontsize=12, fontweight='bold')

for bar, rate in zip(bars, country_by_rate['Cancel_Rate']):
    axes[1].text(bar.get_width() + 0.5, bar.get_y() + bar.get_height()/2, 
                 f'{rate:.1f}%', va='center', fontsize=10)

plt.tight_layout()
plt.savefig('../outputs/figures/country_cancellation.png', dpi=150, bbox_inches='tight')
plt.show()

# Summary
print("="*60)
print("TOP 10 COUNTRIES BY BOOKING VOLUME")
print("="*60)
print(country_stats.to_string())
print(f"\n💡 Insight: Portugal (PRT) có số lượng booking lớn nhất nhưng tỷ lệ huỷ cũng cao")

## 12. Correlation Analysis for Numerical Features

In [ ]:
# Select numerical columns (exclude leakage)
num_cols = [col for col in NUMERICAL_COLUMNS if col in df.columns and col not in LEAKAGE_COLUMNS]
num_cols.append('is_canceled')  # Add target

# Compute correlation matrix
corr_matrix = df[num_cols].corr()

# Plot heatmap
fig, ax = plt.subplots(figsize=(16, 12))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r', 
            center=0, square=True, linewidths=0.5, ax=ax,
            cbar_kws={'shrink': 0.8})
ax.set_title('Correlation Matrix - Numerical Features', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('../outputs/figures/correlation_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

# Correlation with target
print("="*60)
print("CORRELATION WITH TARGET (is_canceled)")
print("="*60)
target_corr = corr_matrix['is_canceled'].drop('is_canceled').sort_values(key=abs, ascending=False)
print("\nTop correlations with is_canceled:")
for col, corr in target_corr.head(10).items():
    direction = "↑" if corr > 0 else "↓"
    print(f"  {col}: {corr:.4f} {direction}")

print(f"\n💡 Insights:")
print("- lead_time có tương quan dương cao nhất với huỷ")
print("- total_of_special_requests có tương quan âm (ít huỷ hơn)")
print("- previous_cancellations có tương quan dương (huỷ nhiều trước = huỷ nhiều sau)")

## 13. Chi-Square Test for Categorical Features

Kiểm định Chi-Square để đánh giá mối quan hệ giữa các biến categorical và target `is_canceled`

In [ ]:
def cramers_v(x, y):
    """Calculate Cramér's V statistic for categorical-categorical association"""
    contingency = pd.crosstab(x, y)
    chi2 = chi2_contingency(contingency)[0]
    n = contingency.sum().sum()
    min_dim = min(contingency.shape) - 1
    return np.sqrt(chi2 / (n * min_dim)) if min_dim > 0 else 0

# Categorical columns (exclude leakage)
cat_cols = [col for col in CATEGORICAL_COLUMNS if col in df.columns and col not in LEAKAGE_COLUMNS]

# Chi-square test results
chi2_results = []
for col in cat_cols:
    contingency = pd.crosstab(df[col], df['is_canceled'])
    chi2, p_value, dof, expected = chi2_contingency(contingency)
    cramers = cramers_v(df[col], df['is_canceled'])
    
    chi2_results.append({
        'Column': col,
        'Chi2_Statistic': chi2,
        'P_Value': p_value,
        'Degrees_Freedom': dof,
        'Cramers_V': cramers,
        'Significant': 'Yes' if p_value < 0.05 else 'No'
    })

chi2_df = pd.DataFrame(chi2_results)
chi2_df = chi2_df.sort_values('Cramers_V', ascending=False)

# Display results
print("="*80)
print("CHI-SQUARE TEST RESULTS")
print("="*80)
print("\nH0: Không có mối quan hệ giữa biến categorical và is_canceled")
print("H1: Có mối quan hệ giữa biến categorical và is_canceled")
print("α = 0.05")
print()
chi2_df

In [ ]:
# Visualize Cramér's V
fig, ax = plt.subplots(figsize=(10, 6))
colors = ['#27ae60' if v > 0.1 else '#f39c12' if v > 0.05 else '#3498db' 
          for v in chi2_df['Cramers_V']]
bars = ax.barh(chi2_df['Column'], chi2_df['Cramers_V'], color=colors)
ax.set_xlabel("Cramér's V (Strength of Association)")
ax.set_title("Association Strength: Categorical Features vs is_canceled", fontsize=12, fontweight='bold')
ax.axvline(x=0.1, color='red', linestyle='--', alpha=0.7, label='Moderate (0.1)')
ax.axvline(x=0.05, color='orange', linestyle='--', alpha=0.7, label='Weak (0.05)')

for bar, v in zip(bars, chi2_df['Cramers_V']):
    ax.text(bar.get_width() + 0.005, bar.get_y() + bar.get_height()/2, 
            f'{v:.3f}', va='center', fontsize=10)

ax.legend()
plt.tight_layout()
plt.savefig('../outputs/figures/chi_square_results.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n💡 Interpretation of Cramér's V:")
print("  - 0.00-0.05: Negligible")
print("  - 0.05-0.10: Weak")  
print("  - 0.10-0.15: Moderate")
print("  - 0.15-0.25: Moderately Strong")
print("  - 0.25+: Strong")
print("\n✓ Tất cả các biến categorical đều có mối quan hệ significant với target (p < 0.05)")

## 14. Data Leakage Detection ⚠️

**QUAN TRỌNG**: Cần loại bỏ các biến có thể gây rò rỉ thông tin từ tương lai vào mô hình.

Các biến cần kiểm tra:
- `reservation_status`: Trạng thái cuối cùng của booking (Check-Out/Canceled/No-Show) → **LEAKAGE TRỰC TIẾP**
- `reservation_status_date`: Ngày cập nhật trạng thái → **LEAKAGE TRỰC TIẾP**
- `assigned_room_type` vs `reserved_room_type`: Có thể có thông tin về cancellation

In [ ]:
# Check reservation_status distribution
if 'reservation_status' in df.columns:
    print("="*60)
    print("⚠️  PHÁT HIỆN DATA LEAKAGE - reservation_status")
    print("="*60)
    print("\nPhân phối reservation_status:")
    status_dist = df['reservation_status'].value_counts()
    print(status_dist)
    
    # Cross-tab với target
    print("\n📊 Crosstab reservation_status vs is_canceled:")
    crosstab = pd.crosstab(df['reservation_status'], df['is_canceled'], margins=True)
    print(crosstab)
    
    print("\n🚨 CẢNH BÁO:")
    print("  - 'Canceled' status HOÀN TOÀN overlap với is_canceled=1")
    print("  - 'Check-Out' và 'No-Show' CHỈ có is_canceled=0")
    print("  → reservation_status là biến LEAKAGE, CẦN LOẠI BỎ!")
    
    # Visualize
    fig, ax = plt.subplots(figsize=(8, 5))
    crosstab_norm = pd.crosstab(df['reservation_status'], df['is_canceled'], normalize='index')
    crosstab_norm.plot(kind='bar', stacked=True, ax=ax, color=['#27ae60', '#e74c3c'])
    ax.set_title('Leakage Evidence: reservation_status vs is_canceled', fontweight='bold')
    ax.set_xlabel('Reservation Status')
    ax.set_ylabel('Proportion')
    ax.legend(title='is_canceled', labels=['Not Canceled', 'Canceled'])
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.savefig('../outputs/figures/leakage_detection.png', dpi=150, bbox_inches='tight')
    plt.show()

In [ ]:
# Additional leakage check: assigned_room_type vs reserved_room_type
if 'assigned_room_type' in df.columns and 'reserved_room_type' in df.columns:
    print("\n" + "="*60)
    print("📋 Kiểm tra: assigned_room_type vs reserved_room_type")
    print("="*60)
    
    # Room type mismatch
    room_mismatch = df['assigned_room_type'] != df['reserved_room_type']
    print(f"\nSố booking có room type khác nhau: {room_mismatch.sum()} ({room_mismatch.mean()*100:.2f}%)")
    
    # Correlation với cancellation
    mismatch_cancel_rate = df[room_mismatch]['is_canceled'].mean()
    match_cancel_rate = df[~room_mismatch]['is_canceled'].mean()
    
    print(f"\n  - Cancel rate khi room MATCH: {match_cancel_rate*100:.2f}%")
    print(f"  - Cancel rate khi room MISMATCH: {mismatch_cancel_rate*100:.2f}%")
    
    print("\n✅ assigned_room_type CÓ THỂ GIỮ LẠI vì:")
    print("   - Được assign KHI customer đến (không biết trước cancellation)")
    print("   - Tuy nhiên, cần cẩn thận với interpretation")

# List all columns to drop
print("\n" + "="*60)
print("📝 DANH SÁCH CÁC CỘT CẦN LOẠI BỎ")
print("="*60)
LEAKAGE_COLUMNS = ['reservation_status', 'reservation_status_date']
print(f"\n🚫 Leakage columns: {LEAKAGE_COLUMNS}")
print("   Lý do: Chứa thông tin về kết quả cuối cùng của booking")

## 15. Outlier Detection 📊

Phát hiện và phân tích các outliers trong các biến numerical để quyết định chiến lược xử lý.

In [ ]:
# Outlier detection using IQR method
def detect_outliers_iqr(df, column):
    """Detect outliers using IQR method"""
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower) | (df[column] > upper)]
    return len(outliers), lower, upper, Q1, Q3

# Numerical columns to check
num_cols_outlier = ['lead_time', 'stays_in_weekend_nights', 'stays_in_week_nights',
                    'adults', 'children', 'babies', 'previous_cancellations',
                    'previous_bookings_not_canceled', 'booking_changes',
                    'days_in_waiting_list', 'adr', 'required_car_parking_spaces',
                    'total_of_special_requests']

outlier_summary = []
for col in num_cols_outlier:
    if col in df.columns:
        count, lower, upper, Q1, Q3 = detect_outliers_iqr(df, col)
        outlier_summary.append({
            'Column': col,
            'Outliers': count,
            'Pct_Outliers': count / len(df) * 100,
            'Q1': Q1,
            'Q3': Q3,
            'Lower_Bound': lower,
            'Upper_Bound': upper,
            'Min': df[col].min(),
            'Max': df[col].max()
        })

outlier_df = pd.DataFrame(outlier_summary)
outlier_df = outlier_df.sort_values('Pct_Outliers', ascending=False)
print("📊 OUTLIER DETECTION SUMMARY (IQR Method)")
print("="*80)
display(outlier_df.round(2))

In [ ]:
# Visualize outliers using boxplots
fig, axes = plt.subplots(3, 4, figsize=(16, 12))
axes = axes.flatten()

for idx, col in enumerate(num_cols_outlier[:12]):
    if col in df.columns:
        ax = axes[idx]
        df.boxplot(column=col, ax=ax)
        ax.set_title(col, fontsize=10)
        ax.set_xlabel('')

# Remove empty subplots if any
for idx in range(len(num_cols_outlier), len(axes)):
    fig.delaxes(axes[idx])

plt.suptitle('Boxplot - Outlier Detection for Numerical Features', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('../outputs/figures/outlier_boxplots.png', dpi=150, bbox_inches='tight')
plt.show()

# Specific analysis for ADR (important feature with many outliers)
print("\n📌 Phân tích chi tiết ADR (Average Daily Rate):")
print(f"   - Min: {df['adr'].min():.2f}")
print(f"   - Max: {df['adr'].max():.2f}")
print(f"   - Median: {df['adr'].median():.2f}")
print(f"   - Mean: {df['adr'].mean():.2f}")

# Check extreme ADR values
extreme_adr = df[df['adr'] > 1000]
negative_adr = df[df['adr'] < 0]
print(f"\n   - Bookings với ADR > 1000: {len(extreme_adr)}")
print(f"   - Bookings với ADR < 0: {len(negative_adr)}")

if len(negative_adr) > 0:
    print("\n   🚨 Giá trị ADR âm là KHÔNG HỢP LỆ - cần xử lý!")

## 16. Summary & Key Findings 📝

### Tổng kết các phát hiện quan trọng từ EDA

In [ ]:
print("="*70)
print("📊 TÓM TẮT EDA - HOTEL BOOKING CANCELLATION PREDICTION")
print("="*70)

print("\n📈 1. THÔNG TIN DATASET:")
print(f"   - Tổng số records: {df.shape[0]:,}")
print(f"   - Số features: {df.shape[1]}")
print(f"   - Target: is_canceled (Binary)")

print("\n📊 2. CLASS IMBALANCE:")
cancel_rate = df['is_canceled'].mean() * 100
print(f"   - Not Canceled: {100-cancel_rate:.2f}%")
print(f"   - Canceled: {cancel_rate:.2f}%")
print(f"   → Imbalanced but manageable (ratio ~1:1.7)")
print(f"   → Solution: SMOTE, class_weight, hoặc stratified sampling")

print("\n🚫 3. MISSING VALUES:")
missing_cols = df.columns[df.isnull().any()].tolist()
print(f"   - Số cột có missing: {len(missing_cols)}")
for col in missing_cols:
    pct = df[col].isnull().mean() * 100
    print(f"     • {col}: {pct:.2f}% missing")
print("   → Solution:")
print("     • company (94%): Drop hoặc tạo binary flag")
print("     • agent (14%): Impute với mode/0 (no agent)")
print("     • country, children: Impute với mode/median")

print("\n⚠️  4. DATA LEAKAGE (CRITICAL):")
print("   CẦN LOẠI BỎ:")
print("   - reservation_status: Contains outcome information")
print("   - reservation_status_date: Temporal leakage")

print("\n📌 5. KEY FEATURES FOR PREDICTION (dựa trên EDA):")
key_features = """
   HIGH IMPORTANCE:
   - lead_time: Longer lead time → Higher cancellation
   - deposit_type: Non Refund có cancellation rate thấp nhất
   - market_segment: Online TA có cancel rate cao nhất
   - previous_cancellations: Strong positive correlation
   - total_of_special_requests: More requests → Less cancellation
   - adr: Higher ADR → Slightly higher cancellation
   
   MODERATE IMPORTANCE:
   - customer_type: Transient có cancel rate cao nhất
   - hotel: City Hotel có cancel rate cao hơn Resort
   - arrival_date_month: Seasonal patterns
   - country: Portugal (PRT) có cancel rate cao
"""
print(key_features)

print("\n🔧 6. PREPROCESSING RECOMMENDATIONS:")
print("   □ Drop: reservation_status, reservation_status_date")
print("   □ Handle missing: company, agent, country, children")
print("   □ Feature Engineering:")
print("     • total_stay = stays_in_weekend_nights + stays_in_week_nights")
print("     • total_guests = adults + children + babies")
print("     • is_repeated_guest flag")
print("     • arrival_date features (month, day_of_week, is_weekend)")
print("   □ Encoding: Label/One-hot for categorical features")
print("   □ Scaling: StandardScaler for numerical features")
print("   □ Outlier treatment: Cap extreme ADR values")

print("\n" + "="*70)
print("✅ EDA COMPLETED - Ready for Preprocessing & Modeling")
print("="*70)

In [ ]:
# Save EDA summary to file
eda_summary = """
# EDA Summary - Hotel Booking Cancellation Prediction
Generated: {timestamp}

## Dataset Overview
- Records: 119,390
- Features: 32
- Target: is_canceled (Binary: 0/1)
- Cancellation Rate: 37.04%

## Data Quality Issues
1. Missing Values:
   - company: 94.31% → Drop or create flag
   - agent: 13.69% → Impute with 0 (no agent)
   - country: 0.41% → Impute with mode
   - children: 0.00% (4 rows) → Impute with 0

2. Data Leakage:
   - MUST DROP: reservation_status, reservation_status_date

3. Outliers:
   - ADR has extreme values (max > 5000, negative values)
   - lead_time has values up to 737 days

## Key Insights
1. Lead Time: Strong positive correlation with cancellation
2. Deposit Type: Non Refund has lowest cancel rate
3. Market Segment: Online TA has highest cancel rate
4. Special Requests: Negative correlation with cancellation
5. Hotel Type: City Hotel has higher cancel rate than Resort

## Recommended Features
- lead_time
- deposit_type
- market_segment
- previous_cancellations
- total_of_special_requests
- adr
- customer_type
- is_repeated_guest
- Required feature engineering for temporal patterns
"""

from datetime import datetime
with open('../outputs/eda_summary.txt', 'w', encoding='utf-8') as f:
    f.write(eda_summary.format(timestamp=datetime.now().strftime('%Y-%m-%d %H:%M:%S')))
print("✅ EDA summary saved to outputs/eda_summary.txt")